<a href="https://colab.research.google.com/github/watchstep/rerank/blob/dev/exllamav2_beir_rerank_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [Promptagator: Few-shot Dense Retrieval From 8 Examples](https://arxiv.org/abs/2209.11755)
- [InPars: Data Augmentation for Information Retrieval using Large Language Models.](https://arxiv.org/abs/2202.05144)

- https://github.com/eric88525/UDADF/tree/master
- https://blog.vespa.ai/improving-zero-shot-ranking-with-vespa-part-two/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install exllamav2  beir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 

In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
!git clone -b 4.0bpw https://huggingface.co/turboderp/Mistral-7B-instruct-exl2

Cloning into 'Mistral-7B-instruct-exl2'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), 623.05 KiB | 2.01 MiB/s, done.


In [ ]:
import os, sys, argparse
import json

p_prompt = f"""
Passage: <P> Given the provided passage, generate 3 similar passages on related topic: <T>
"""

q_prompt = """
<P> Review the given passages and answer a specific and detailed query. {'Query: Your query here.'}”
"""

parser = argparse.ArgumentParser(description = "Prompting with ExLlamaV2")
parser.add_argument("--dataset", type = str, default ='nq')
parser.add_argument("--seed", type = int, default = 2023)
parser.add_argument("--topk", type=int, default=10)
parser.add_argument("--model_dir", type=str, default="/content/Mistral-7B-instruct-exl2")
parser.add_argument("--p_prompt", type=str, default=p_prompt)
parser.add_argument("--q_prompt", type=str, default=q_prompt)

args = parser.parse_args([])

In [ ]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{args.dataset}.zip"
data_path = util.download_and_unzip(url, '/content')

corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

/usr/local/lib/python3.10/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/content/nq.zip:   0%|          | 0.00/475M [00:00<?, ?iB/s]

  0%|          | 0/2681468 [00:00<?, ?it/s]

In [ ]:
# download & setup Elasticsearch
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
%%bash

ps -ef | grep elasticsearch

root        2906    2901  0 06:31 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasti
daemon      2907    2906 39 06:31 ?        00:00:17 /content/elasticsearch-7.9.2/jdk/bin/java -Xshar
root        3322    3320  0 06:32 ?        00:00:00 grep elasticsearch


In [ ]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "dfdcb6919169",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "9fLaMVCpR9q87we_5jb0Zw",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

hostname = "localhost"
index_name = "nq"
initialize = True

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
retriever = EvaluateRetrieval(model)

results = retriever.retrieve(corpus, queries)
# ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
output = {'retrieval': EvaluateRetrieval.evaluate(qrels, results, retriever.k_values)}

que: 100%|██████████| 27/27 [09:50<00:00, 21.88s/it]


In [ ]:
# save retrieved
with open(f'/content/drive/MyDrive/{args.dataset}_bm25_es_retrieved.json', 'w') as f:
  json.dump(results, f, indent=4)

# save output tmp
with open(f'/content/drive/MyDrive/{args.dataset}_eval_output.json', 'w') as f:
  json.dump(output, f, indent=4)

In [ ]:
def save_json(results, file_name:str):
  json_path = f'/content/drive/MyDrive/{args.dataset}_{file_name}.json'
  with open(json_path, 'w') as f:
    json.dump(results, f, indent=4)

def read_json(json_path:str):
  with open(json_path, 'r') as f:
    data = f.read()
  output = json.loads(data)

  return output

In [ ]:
results = read_json(f'/content/drive/MyDrive/{args.dataset}_bm25_es_retrieved_.json')
output = read_json(f'/content/drive/MyDrive/{args.dataset}_eval_ouput.json')

In [ ]:
from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer,
)

from exllamav2.generator import (
    ExLlamaV2BaseGenerator,
    ExLlamaV2Sampler
)

# from beir.reranking import Rerank
from typing import List, Tuple
import torch
import torch.nn.functional as F

In [ ]:
# https://github.com/beir-cellar/beir/blob/main/beir/reranking/rerank.py
from typing import Dict, List

class Rerank:
  def __init__(self, model, batch_size: int = 128, **kwargs):
    self.model = model
    self.batch_size = batch_size
    self.rerank_results = {}

  def rerank(self,
             corpus: Dict[str, Dict[str, str]],
             queries: Dict[str, str],
             results: Dict[str, Dict[str, float]],
             top_k: int) -> Dict[str, Dict[str, float]]:

    sentence_pairs, pair_ids = [], []

    for query_id in results:
      if len(results[query_id]) > top_k:
        for (doc_id, _) in sorted(results[query_id].items(), key=lambda item: item[1], reverse=True)[:top_k]:
          pair_ids.append([query_id, doc_id])
          corpus_text = {'title': corpus[doc_id].get("title", "").strip(), 'text': corpus[doc_id].get("text", "").strip()}
          sentence_pairs.append([queries[query_id], corpus_text])

      else:
        for doc_id in results[query_id]:
          pair_ids.append([query_id, doc_id])
          corpus_text = {'title': corpus[doc_id].get("title", "").strip(), 'text': corpus[doc_id].get("text", "").strip()}
          # corpus_text = (corpus[doc_id].get("title", "") + " " + corpus[doc_id].get("text", "")).strip()
          sentence_pairs.append([queries[query_id], corpus_text])

    #### Starting to Rerank using cross-attention
    rerank_scores = [float(score) for score in self.model.predict(sentence_pairs, batch_size=self.batch_size)]

    #### Reranking results
    self.rerank_results = {query_id: {} for query_id in results}
    for pair, score in zip(pair_ids, rerank_scores):
      query_id, doc_id = pair[0], pair[1]
      self.rerank_results[query_id][doc_id] = score

    return self.rerank_results

In [ ]:
# https://github.com/beir-cellar/beir/wiki/Evaluate-your-custom-model
# https://github.com/Muennighoff/sgpt/blob/main/crossencoder/beir/sgptce.py
# https://github.com/beir-cellar/beir/blob/main/examples/retrieval/evaluation/reranking/evaluate_bm25_ce_reranking.py
# https://github.com/beir-cellar/beir/blob/main/beir/reranking/models/mono_t5.py

class ExLlamaV2Reranker:
  def __init__(self, p_prompt=args.p_prompt, q_prompt=args.q_prompt, model_dir=args.model_dir, seed=args.seed, max_new_tokens=400, **kwargs):
    self.p_prompt = p_prompt
    self.q_prompt = q_prompt
    self.seed = seed
    self.max_new_tokens = max_new_tokens

    self.config = ExLlamaV2Config()
    self.config.model_dir = model_dir
    self.config.prepare()

    self.model = ExLlamaV2(self.config)
    if not self.model.loaded:
      self.cache = ExLlamaV2Cache(self.model, lazy = True)
      self.model.load_autosplit(self.cache)

    self.tokenizer = ExLlamaV2Tokenizer(self.config)
    self.generator = ExLlamaV2BaseGenerator(self.model, self.cache, self.tokenizer)
    self.generator.warmup()

    self.settings = ExLlamaV2Sampler.Settings()
    self.settings.temperature = 0.6
    self.settings.top_k = 50
    self.settings.top_p = 0.9
    self.settings.token_repetition_penalty = 1.15
    self.settings.disallow_tokens(self.tokenizer, [self.tokenizer.eos_token_id])

  def predict(self, sentences: List[Tuple[str,str]], batch_size: int, **kwags) -> List[float]:
    scores = []
    for query, psg in sentences:
      self.p_prompt = self.p_prompt.replace('<P>', psg['text']).replace('<T>', psg['title']).strip()
      generated = self.generator.generate_simple(self.p_prompt, self.settings, self.max_new_tokens, seed=self.seed)
      self.q_prompt = self.q_prompt.replace('<P>', generated).strip()
      with torch.inference_mode():
        input_ids = self.tokenizer.encode(self.q_prompt)
        # input_ids = input_ids.shape[-1]
        # self.cache.current_seq_len = 0
        logits = self.model.forward(input_ids[:, -1:], self.cache)
        logits = logits[:, :-1, :]
        logits = logits.float() + 1e-10
        log_probs = F.log_softmax(logits, dim=-1)

      scores.append(log_probs)

    assert len(scores) == len(sentences)
    return scores


In [ ]:
reranker = Rerank(ExLlamaV2Reranker(), batch_size=128)
rerank_results = reranker.rerank(corpus, queries, results, top_k=10)
# rerank_results = reranker.rerank(corpus, queries, results, top_k=100)

In [ ]:
# save output
output['reranking'] = EvaluateRetrieval.evaluate(qrels, rerank_results, retriever.k_values)

with open(f'/content/{args.dataset}_eval_ouput.json', 'w') as f:
  json.dump(output, f, indent=4)

In [ ]:
import random, logging

top_k = 10

query_id, ranking_scores = random.choice(list(rerank_results.items()))
scores_sorted = sorted(ranking_scores.items(), key=lambda item: item[1], reverse=True)
logging.info("Query : %s\n" % queries[query_id])

for rank in range(top_k):
  doc_id = scores_sorted[rank][0]
  # Format: Rank x: ID [Title] Body
  logging.info("Rank %d: %s [%s] - %s\n" % (rank+1, doc_id, corpus[doc_id].get("title"), corpus[doc_id].get("text")))

### Example

In [ ]:
te_text = corpus['doc787']['text']
te_title = corpus['doc787']['title']

In [ ]:
prompt = f"""
Passage: {te_text} Please generate 3 passages akin to the given passages, focusing on {te_title}
"""

# max_new_tokens = 400

output = generator.generate_simple(prompt, settings, seed = 2023)
print(output)

In [ ]:
prompt = f"""
Passage: {te_text} Given the provided passage, generate 3 similar passages on related title {te_title}
"""

max_new_tokens = 400

output = generator.generate_simple(prompt, settings, max_new_tokens, seed = 2023)
print(output)

In [ ]:
prompt2 = f"{output} Review the given passages and answer a specific and detailed query. {'Query: Your query here.'}"
max_new_tokens = 30

output2 = generator.generate_simple(prompt2, settings, max_new_tokens, seed = 2023)
print(output2)

In [ ]:
class NQDataset:
  def __init__(self, corpus, queries):
    self.corpus = corpus
    self.queries = queries
    self.qids = list(queries.keys())

  def __len__(self):
    return len(self.qids)

  def __getitem__(self, idx):
    query = self.queries[self.qids[idx]]
    return query

In [ ]:
with torch.inference_mode():
  cache = ExLlamaV2Cache(model)

  input_ids1 = tokenizer.encode(prompt1)
  logits = model.forward(input_ids, cache)


  output1 = generator.generate()

In [ ]:
input_ids = tokenizer.encode(prompt1)
prompt_tokens = input_ids.shape[-1]

output = generator.generate(batch["input_ids"].cuda(), settings, seed=2023)

decode